In [1]:
import openai
import json
import sys
import os
from dotenv import load_dotenv

In [2]:
FIREWORKS_API_KEY = os.getenv("FIREWORKS_API_KEY")
E2B_API_KEY = os.getenv("E2B_API_KEY")

print(os.getenv('FIREWORKS_API_KEY'))

fw_3ZfUrfHdKvGhXmgDTrSwdLC3


In [3]:
SYSTEM_PROMPT = """
## your job & context
you are a python data scientist. you are given tasks to complete and you run python code to solve them.
You DO NOT MAKE SYNTAX MISTAKES OR FORGET ANY IMPORTS
- the python code runs in jupyter notebook.
- every time you call `execute_python` tool, the python code is executed in a separate cell. it's okay to multiple calls to `execute_python`.
- display visualizations using matplotlib or any other visualization library directly in the notebook. don't worry about saving the visualizations to a file.
- you have access to the internet and can make api requests.
- you also have access to the filesystem and can read/write files.
- you can install any pip package (if it exists) if you need to but the usual packages for data analysis are already preinstalled.
- you can run any python code you want, everything is running in a secure sandbox environment.
"""

USER_PROMPT = "Plot a 3D chart of sin x cos y please."

client = openai.OpenAI(
    base_url = "https://api.fireworks.ai/inference/v1",
    api_key = FIREWORKS_API_KEY
)

tools = [
    {
        "type": "function",
        "function": {
            "name": "execute_python",
            "description": "Execute python code in a Jupyter notebook cell and returns any result, stdout, stderr, display_data, and error.",
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {
                        "type": "string",
                        "description": "The python code to execute in a single cell.",
                    },
                },
                "required": ["code"],
            },
        },
        },
]

In [4]:
def code_interpret(e2b_code_interpreter, code):
  print("Running code interpreter...")
  exec = e2b_code_interpreter.notebook.exec_cell(
    code,
    # Stream stdout and stderr from the Code Interpreter
    on_stdout=lambda stdout: print("[Code Interpreter]", stdout),
    on_stderr=lambda stderr: print("[Code Interpreter]", stderr),
    # You can also stream code execution results
    # on_result=...
  )

  if exec.error:
    print("[Code Interpreter ERROR]", exec.error)
  else:
    return exec

In [5]:
def chat(e2b_code_interpreter, user_message):
  print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")
  
  messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": user_message}
  ]

  response = client.chat.completions.create(
    model = "accounts/fireworks/models/firefunction-v2",
    messages=messages,
    tools=tools,
    tool_choice="auto"
  )
  for choice in response.choices:
    if choice.message.tool_calls and len(choice.message.tool_calls) > 0:
      for tool_call in choice.message.tool_calls:
        if tool_call.function.name == "execute_python":
          if "code" in tool_call.function.arguments:
            args = json.loads(tool_call.function.arguments)
            code = args["code"]          
            print("CODE TO RUN")
            print(code)
            execution_output = code_interpret(e2b_code_interpreter, code)
            return execution_output
    else:
      print("Answer:", choice.message.content)

In [6]:
from e2b_code_interpreter import CodeInterpreter

with CodeInterpreter(api_key=E2B_API_KEY) as code_interpreter:
  execution_output = chat(
    code_interpreter,
    USER_PROMPT,
  )

  if execution_output == None:
    print("No output from the code interpreter. Did the LLM generate any code?")
    sys.exit(1)

  # Access stdout
  print("Stdout:", execution_output.logs.stdout)
  # Access stderr
  print("Stderr:", execution_output.logs.stderr)

  # Access any runtime errors
  print("AI-generated Python runtime error:", execution_output.error)

  # Access any results of code execution - charts, interpreter last line, images, etc.
  print("Results:", execution_output.results)
  if len(execution_output.results) == 0:
    sys.exit(0)

execution_output.results[0]

Failed to acquire sandbox


UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json; charset=utf-8', 'date': 'Sat, 05 Oct 2024 19:28:50 GMT', 'Content-Length': '157', 'via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"code":401,"message":"Invalid API key, please visit https://e2b.dev/docs?reason=sdk-missing-api-key to get your API key. authorization header is malformed"}
